In [181]:
import numpy as np
import pandas as pd
%pip install matplotlib
import matplotlib.pyplot as plt
%pip install seaborn
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [182]:
data = pd.read_csv('./data.csv')
data[data.columns] = data[data.columns].astype('Int64')

In [183]:
data = data.drop('Final_EDSS', axis=1)
data

,Unnamed: 0,Gender,Age,Schooling,Breastfeeding,Varicella,Initial_Symptom,Mono_or_Polysymptomatic,Oligoclonal_Bands,LLSSEP,ULSSEP,VEP,BAEP,Periventricular_MRI,Cortical_MRI,Infratentorial_MRI,Spinal_Cord_MRI,Initial_EDSS,group
0,0,1,34,20,1,1,2,1,0,1,1,0,0,0,1,0,1,1,1
1,1,1,61,25,3,2,10,2,1,1,0,1,0,0,0,0,1,2,1
2,2,1,22,20,3,1,3,1,1,0,0,0,0,0,1,0,0,1,1
3,3,2,41,15,1,1,7,2,1,0,1,1,0,1,1,0,0,1,1
4,4,2,34,20,2,1,6,2,0,1,0,0,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,268,2,31,8,3,1,9,2,0,0,0,0,0,0,0,0,0,<NA>,2
269,269,1,21,15,3,3,5,2,1,0,0,0,0,0,0,0,1,<NA>,2
270,270,2,19,12,3,3,13,2,0,1,1,1,0,0,0,0,1,<NA>,2
271,271,2,32,15,3,3,15,2,1,1,1,1,0,1,1,1,0,<NA>,2


### Modelo 1 imputando valores en initial-edss - random forest

In [184]:
data['group'] = data['group'].replace(2, 0)

In [185]:
data['imputado?'] = 0

# valores faltantes en initial_edss
indices_faltantes = data[data['Initial_EDSS'].isnull()].index
num_missing = len(indices_faltantes)
print(num_missing) #cuantos nan hay

148


In [186]:
# Calcular la cantidad de valores a asignar a cada categoría
quarter = num_missing // 4
remainder = num_missing % 4

# Crear una nueva columna para indicar si se imputó un valor
data['imputado?'] = 0

# Imputar valores a los índices faltantes
data.loc[indices_faltantes[:quarter], 'Initial_EDSS'] = 1
data.loc[indices_faltantes[quarter:2*quarter], 'Initial_EDSS'] = 2
data.loc[indices_faltantes[2*quarter:], 'Initial_EDSS'] = 3
data.loc[indices_faltantes, 'imputado?'] = 1
# Verificar que todos los valores faltantes hayan sido asignados
assert data['Initial_EDSS'].isnull().sum() == 0
data

,Unnamed: 0,Gender,Age,Schooling,Breastfeeding,Varicella,Initial_Symptom,Mono_or_Polysymptomatic,Oligoclonal_Bands,LLSSEP,ULSSEP,VEP,BAEP,Periventricular_MRI,Cortical_MRI,Infratentorial_MRI,Spinal_Cord_MRI,Initial_EDSS,group,imputado?
0,0,1,34,20,1,1,2,1,0,1,1,0,0,0,1,0,1,1,1,0
1,1,1,61,25,3,2,10,2,1,1,0,1,0,0,0,0,1,2,1,0
2,2,1,22,20,3,1,3,1,1,0,0,0,0,0,1,0,0,1,1,0
3,3,2,41,15,1,1,7,2,1,0,1,1,0,1,1,0,0,1,1,0
4,4,2,34,20,2,1,6,2,0,1,0,0,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,268,2,31,8,3,1,9,2,0,0,0,0,0,0,0,0,0,3,0,1
269,269,1,21,15,3,3,5,2,1,0,0,0,0,0,0,0,1,3,0,1
270,270,2,19,12,3,3,13,2,0,1,1,1,0,0,0,0,1,3,0,1
271,271,2,32,15,3,3,15,2,1,1,1,1,0,1,1,1,0,3,0,1


In [187]:
# Separar las características y la etiqueta
X = data.drop('group', axis=1)
y = data['group']

In [188]:
# Dividir los datos en entrenamiento, validación y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [189]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [192]:
y_pred = rf.predict(X_val)
exactitud = accuracy_score(y_val, y_pred)
print("exactitud:", exactitud)

exactitud: 1.0
